In [ ]:
import torch
def get_mask(low_rate, high_rate):
    mask = torch.zeros(224,113)
    cy, cx = 224//2, 0
    rh, rw = int(high_rate*(224)/2), int(high_rate*(113))

    mask[cy-rh:cy+rh, :cx+rw]=1

    rh, rw = int(low_rate*(224)/2), int(low_rate*(113))

    mask[cy-rh:cy+rh, :cx+rw]=0

    return mask

In [ ]:
import torch
import torch.nn as nn
import torch.fft
from PIL import Image
import torchvision.transforms as transforms

from torchvision.utils import save_image

image_path = 'input.jpg'
x = Image.open(image_path).convert('RGB')
loader = transforms.ToTensor()
x = loader(x)
print(x.size())


fft = torch.fft.rfftn(x, dim=(-2,-1), norm='ortho')
print(fft.size())


fft = torch.roll(fft,(224//2, 0),dims=(1,2))
frequency = torch.abs(fft)

print(frequency.size())

save_image(tensor=frequency, fp='frequency.jpg')

mask = get_mask(0.5, 1)
save_image(tensor=mask, fp='mask.jpg')

fft_low = torch.mul(mask, fft)
print(fft_low.size())
frequency_low = torch.abs(fft_low)
save_image(tensor=frequency_low, fp='frequency_low.jpg')
fft_low = torch.roll(fft_low,(224//2, 0),dims=(1,2))

output = torch.fft.irfftn(fft_low, dim=(-2,-1), norm='ortho')
save_image(tensor=output, fp='output.jpg')
